In [13]:
import requests
from bs4 import BeautifulSoup as bs
import pymongo
import  re
import json


with open('entities.json', encoding='utf-8') as f:
    templates = json.load(f)

headers = {'accept': '*/*',
           'user-agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 YaBrowser/20.3.0.2220 Yowser/2.5 Safari/537.36'}
db_name = templates['db_name']
coll_name = templates['coll_name']
url = templates['url']
q_type = templates['q_type']
connect = templates['connect']
url = templates['url']
#connect = 'DESKTOP-Q4QO50O'
#port = 27018
#db_name = 'stackoverflow_data'
#coll_name = 'raw_data'
#q_type = 'bigdata'

In [14]:
import requests
from bs4 import BeautifulSoup as bs
import pymongo
import  re
import json


def db_format_element(advert):
    doc = {
               "id":advert[0],
               "head": advert[1],
               "votes_count": advert[2],
               "answers_count": advert[3],
               "tag_list": advert[4],
               "q_type": advert[5],
               "link": advert[6],
               "text": advert[7]
          }
    return doc


def parser(base_url, connect, header, db_name, coll_name, q_type, is_replace=False):
    
    conn = pymongo.MongoClient(connect)
    db = conn[db_name]
    coll = db[coll_name]
    if is_replace:
        coll.delete_many({"q_type": q_type})
    
    rows = []
    page_counter = 112
    url = base_url + str(page_counter)
    session = requests.Session()
    request = session.get(url, headers=header, cookies={'from-my': 'browser'})
    if request.status_code == 200:
        last_page = False
        while not(last_page):
            soup = bs(request.content, 'html.parser')
            divs = soup.find_all('div', attrs={'class': 'question-summary'})
            if divs == []:
                print("website is down for parsing?")
            for div in divs:
                head = div.find('a', attrs={'class': 'question-hyperlink'}).text
                qlink = 'https://stackoverflow.com' + div.find('a', attrs={'class': 'question-hyperlink'}).attrs['href']
                votes_count = div.find('span', attrs={'class': 'vote-count-post'}).text 
                answers_count = div.find('div', attrs={'class': 'status answered-accepted'})
                if answers_count is None:
                    answers_count = div.find('div', attrs={'class': 'status unanswered'}) #status answered
                    if answers_count is None:
                        answers_count = div.find('div', attrs={'class': 'status answered'})    
                answers_count = answers_count.text[1]
                tags = div.find_all('a', attrs={'class': 'post-tag'})
                tag_list=""
                for el in tags:
                    tag_list += el.text + ", "
                tag_list = tag_list[:-2]
                qid = div.attrs["id"].split("-")[-1]
                
                request_2 = session.get(qlink, headers=header)
                soup_2 = bs(request_2.content, 'html.parser')                
                text = soup_2.find('div', attrs={'class': 'post-text'}).text
                
                el_db = db_format_element([qid, head, votes_count, answers_count, tag_list, q_type, qlink, text])
                coll.insert_one(el_db)
            nextPage = soup.find("a", {"rel": "next"})
            print(str(page_counter / 141) + "%")
            print(base_url)
            print(url)
            if nextPage is not None:
                page_counter += 1
                url = base_url + str(page_counter)
                request = session.get(url, headers=header)
                print(1)
            else:
                last_page = True
                print(2)
    else:
        print('error')

In [15]:
parser(url, connect, headers, db_name, coll_name, q_type)

0.7943262411347518%
https://stackoverflow.com/questions/tagged/bigdata?tab=newest&pagesize=50&page=
https://stackoverflow.com/questions/tagged/bigdata?tab=newest&pagesize=50&page=112
1
0.8014184397163121%
https://stackoverflow.com/questions/tagged/bigdata?tab=newest&pagesize=50&page=
https://stackoverflow.com/questions/tagged/bigdata?tab=newest&pagesize=50&page=113
1
0.8085106382978723%
https://stackoverflow.com/questions/tagged/bigdata?tab=newest&pagesize=50&page=
https://stackoverflow.com/questions/tagged/bigdata?tab=newest&pagesize=50&page=114
1
0.8156028368794326%
https://stackoverflow.com/questions/tagged/bigdata?tab=newest&pagesize=50&page=
https://stackoverflow.com/questions/tagged/bigdata?tab=newest&pagesize=50&page=115
1
0.8226950354609929%
https://stackoverflow.com/questions/tagged/bigdata?tab=newest&pagesize=50&page=
https://stackoverflow.com/questions/tagged/bigdata?tab=newest&pagesize=50&page=116
1
0.8297872340425532%
https://stackoverflow.com/questions/tagged/bigdata?tab=

In [6]:
sq = "asdasd, dsad, sdsa, "
sq = sq[:-2]

In [7]:
sq

'asdasd, dsad, sdsa'